In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

#torch.set_num_threads(6)
torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
def sample(pts_silent, pts_implicit1, R, C, sigma):
    while True:
        # We first pick a random index in pts_silent and get the normal position
        # We substract 1 to the possible index so we can easily get the tangent vector
        # ind = np.asscalar(np.random.choice(pts_silent.shape[0] - 1, 1))
        # tan = (pts_silent[ind + 1] - pts_silent[ind]).numpy()
        # norm = np.array([tan[1], -tan[0]])
        # norm /= np.linalg.norm(norm)
        # norm *= -150.
        # cotan = torch.zeros_like(pts_silent)
        # cotan[ind] = torch.tensor(norm)

        #cotan = 20.*torch.randn_like(pts_silent)
        cotan = 1.*torch.randn_like(pts_silent)
        #cotan = torch.zeros_like(pts_silent)
        # cotan[:10, 0] = -10.
        # cotan[120:130, 0] = -10.
        #cotan[pts_silent[:, 0] <= 0, 0] = -2.

        # val_left = -1.
        # val_right = 1.
        # cotan[30:39, 1] = val_left
        # cotan[212:221, 1] = -val_left
        # cotan[92:101, 1] = val_right
        # cotan[149:158, 1] = -val_right

        # plt.plot(pts_silent[:, 0].numpy(), pts_silent[:, 1].numpy())
        # plt.quiver(pts_silent[:, 0].numpy(), pts_silent[:, 1].numpy(), cotan[:, 0].numpy(), cotan[:, 1].numpy())
        # plt.show()

        silent = dm.DeformationModules.SilentLandmarks.build_from_points(pts_silent)
        silent.manifold.fill_cotan(cotan.view(-1).requires_grad_())
        # implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C[:, :, 0].unsqueeze(2), sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
        # implicit1_2 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C[:, :, 1].unsqueeze(2), sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
        implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C, sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
        with torch.autograd.no_grad():
            dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 10, 'euler')
            sample = silent.manifold.gd.detach().view(-1, 2)
            sample = sample - torch.mean(sample, dim=0)
            print(torch.norm(sample).item())
        if torch.norm(sample).item() > torch.norm(pts_silent)*1.1 or torch.norm(sample).item() < torch.norm(pts_silent)/10. or math.isnan(torch.norm(sample).item()):
            continue
        else:
            return sample

In [ ]:
n_pts = 750
vertical_scaling = 10.
ellipse = torch.stack([torch.cos(torch.linspace(0., 2.*math.pi, n_pts)),
                     vertical_scaling*torch.sin(torch.linspace(0., 2.*math.pi, n_pts))], dim=1)

aabb_ellipse = dm.Utilities.AABB.build_from_points(ellipse)

density = 2

pts_implicit1 = dm.Utilities.fill_area_uniform_density(dm.Utilities.area_shape, aabb_ellipse, density, shape=ellipse, side=1)


In [ ]:
%matplotlib qt5
plt.plot(ellipse.numpy()[:-1, 0], ellipse.numpy()[:-1, 1])
plt.plot(pts_implicit1.numpy()[:, 0], pts_implicit1.numpy()[:, 1], '.')
plt.axis('equal')
plt.show()

In [ ]:
C = torch.ones(pts_implicit1.shape[0], 2, 1)
C[:, 1 , 0] = 1.*(pts_implicit1[:, 0]-torch.min(pts_implicit1[:, 0])) + 0.5

%matplotlib qt5
ax = plt.subplot()
plt.axis(aabb_ellipse.get_list())
#plt.axis('equal')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, scale=0.1)
plt.show()

In [ ]:
%matplotlib qt5
ax_left = plt.subplot(1, 2, 1)
dm.Utilities.plot_C_ellipse(ax_left, pts_implicit1, C, c_index=0, scale=0.1, color='red')
dm.Utilities.plot_C_arrow(ax_left, pts_implicit1, C, c_index=0, scale=0.1, color='black', zorder=3, mutation_scale=10)
plt.axis(aabb_ellipse.get_list())
plt.axis('equal')

# ax_right = plt.subplot(1, 2, 2)
# dm.Utilities.plot_C_ellipse(ax_right, pts_implicit1, C, c_index=1, scale=0.1, color='blue')
# dm.Utilities.plot_C_arrow(ax_right, pts_implicit1, C, c_index=1, scale=0.1, color='black', zorder=3, mutation_scale=10)
# plt.axis(aabb_ellipse.get_list())
# plt.axis('equal')

plt.show()

In [ ]:
angles = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(a) for a in angles])

In [ ]:
my_sample = sample(ellipse, pts_implicit1, R, C, 3./math.sqrt(density))

ax = plt.subplot()
plt.plot(ellipse[:, 0].numpy(), ellipse[:, 1].numpy(), '--')
plt.plot(my_sample[:, 0].numpy(), my_sample[:, 1].numpy())
plt.axis(aabb_ellipse.get_list())
plt.axis('equal')
plt.show()

In [ ]:
sample_count = 100
samples = [ellipse.tolist()]
for i in range(sample_count):
    print("Generating sample", i)
    samples.append(sample(ellipse, pts_implicit1, R, C, 3./math.sqrt(density)).tolist())

In [ ]:
nb_plot_row = int(math.sqrt(sample_count)) + 1
nb_plot_col = int(math.sqrt(sample_count)) + 1
cur_plot_index = 0
cont = True
while cont:
    for i in range(nb_plot_row):
        for j in range(nb_plot_col):
            if cur_plot_index < len(samples):
                plt.subplot(nb_plot_row, nb_plot_col, nb_plot_row*i+j+1)
                plt.axis(aabb_ellipse.get_list())
                plt.axis('equal')
                plt.plot(np.asarray(samples[cur_plot_index])[:, 0], np.asarray(samples[cur_plot_index])[:, 1])
                cur_plot_index = cur_plot_index + 1
            else:
                #plt.show()
                cont = False
                break

    plt.show()

In [ ]:
pickle.dump({'c_gt_pos': pts_implicit1, 'c_gt': C, 'dataset': samples}, open("../../data/deformed_ellipses.pkl", 'wb'))